## Project initialization and import libraries

In [1]:
from time import time

import numpy as np

import matplotlib.pyplot as plt
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM


Using TensorFlow backend.


## Import training and test data from csv file

In [2]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')